In [3]:
# 📦 Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import plotly.express as px
from google.colab import files

# 📤 Upload the dataset file
uploaded = files.upload()  # Select 'Online Retail.xlsx' after running this

# 📄 Load the Excel File
df = pd.read_excel('Online Retail.xlsx')

# 🧹 Data Cleaning
df.dropna(inplace=True)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# 🧠 RFM Feature Creation
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# 🔎 Scaling the Features
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# 🔍 Finding Best Number of Clusters Using Silhouette Score
best_k = 0
best_score = -1
best_model = None

for k in range(2, 11):
    model = KMeans(n_clusters=k, random_state=42)
    labels = model.fit_predict(rfm_scaled)
    score = silhouette_score(rfm_scaled, labels)
    print(f"K={k} --> Silhouette Score: {score:.4f}")

    if score > best_score:
        best_k = k
        best_score = score
        best_model = model

# ✅ Final Clustering
rfm['Cluster'] = best_model.labels_
print(f"\n✅ Best K: {best_k} with Silhouette Score: {best_score:.4f}")

# 📊 3D Cluster Visualization
fig = px.scatter_3d(rfm, x='Recency', y='Frequency', z='Monetary', color='Cluster',
                    title=f'Customer Segmentation (K={best_k})',
                    labels={'Recency': 'Recency', 'Frequency': 'Frequency', 'Monetary': 'Monetary'})
fig.show()

# 📋 Cluster Summary Table
summary = rfm.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(2)

print("\n📈 Cluster Summary:\n")
print(summary)


Saving Online Retail.xlsx to Online Retail (1).xlsx


<ipython-input-3-3aaf78f25f6d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Quantity'] * df['UnitPrice']


K=2 --> Silhouette Score: 0.5604
K=3 --> Silhouette Score: 0.5853
K=4 --> Silhouette Score: 0.6162
K=5 --> Silhouette Score: 0.6171
K=6 --> Silhouette Score: 0.5983
K=7 --> Silhouette Score: 0.5172
K=8 --> Silhouette Score: 0.4909
K=9 --> Silhouette Score: 0.4840
K=10 --> Silhouette Score: 0.4250

✅ Best K: 5 with Silhouette Score: 0.6171



📈 Cluster Summary:

        Recency Frequency   Monetary      
           mean      mean       mean count
Cluster                                   
0         43.81      3.69    1358.77  3060
1        248.47      1.55     480.42  1063
2          1.50    135.83   58381.12     6
3         15.72     22.30   13534.00   203
4          7.67     42.83  190863.46     6
